In [4]:
from google.colab import files
uploaded = files.upload()

Saving games_march2025_cleaned.csv to games_march2025_cleaned.csv


In [40]:
import pandas as pd

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename, low_memory=False)
print(df.shape)
df.head()

(89618, 47)


,appid,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,discount,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,879,5174,350,0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",86,8632939,82,96473
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",59,2513842,68,16720
2,570,Dota 2,2013-07-09,0,0.00,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,1536,898,892,0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",81,2452595,80,29366
3,271590,Grand Theft Auto V Legacy,2015-04-13,17,0.00,0,"When a young street hustler, a retired bank ro...","When a young street hustler, a retired bank ro...",Grand Theft Auto V for PC offers players the o...,NaN,...,771,7101,74,0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",87,1803832,92,17517
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,3.99,9,Edition Comparison Ultimate Edition The Tom Cl...,“One of the best first-person shooters ever ma...,"Tom Clancy's Rainbow Six® Siege is an elite, t...",NaN,...,682,2434,306,80,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",84,1168020,76,12608


In [41]:
import pandas as pd
import numpy as np
import ast

cols_to_drop = [
    "name",
    "detailed_description",
    "about_the_game",
    "short_description",
    "reviews",
    "header_image",
    "website",
    "support_url",
    "support_email",
    "metacritic_url",
    "notes",
    "screenshots",
    "movies",
    "packages",
    "release_date",
    "metacritic_score",
    "pct_pos_total",
    "pct_pos_recent",
    "num_reviews_recent",
    "score_rank",
    "average_playtime_forever",
    "average_playtime_2weeks",
    "median_playtime_forever",
    "median_playtime_2weeks",
    "discount",
]

cols_to_drop = [c for c in cols_to_drop if c in df.columns]
df = df.drop(columns=cols_to_drop)
print("After column drop:", df.shape)

# windows /mac/ linux: convert True/False to 1/0
for col in ["windows", "mac", "linux"]:
    if col in df.columns:
        df[col] = df[col].map({True: 1, False: 0, "TRUE": 1, "FALSE": 0}).fillna(0).astype(int)

# required_age: replace -1 with 0
if "required_age" in df.columns:
    df["required_age"] = df["required_age"].replace(-1, 0)

# num_reviews_total: treat -1 as missing
if "num_reviews_total" in df.columns:
    df["num_reviews_total"] = df["num_reviews_total"].replace(-1, np.nan)


# estimated_owners: convert "A - B" string to midpoint (A+B)/2

def owners_mid(s):
    if pd.isna(s):
        return np.nan
    s = str(s)
    if "-" not in s:
        return np.nan
    low, high = s.split("-")
    try:
        low = int(low.strip().replace(",", ""))
        high = int(high.strip().replace(",", ""))
    except ValueError:
        return np.nan
    return (low + high) / 2.0

if "estimated_owners" in df.columns:
    tmp = df["estimated_owners"].apply(owners_mid)
    mask = tmp.notna() & (tmp > 0)
    before = df.shape[0]
    df = df[mask].copy()
    df["estimated_owners"] = tmp[mask].astype(float)
    print(f"Rows after cleaning estimated_owners: {before} -> {df.shape[0]}")

# 5. Drop rows where positive + negative == 0
if {"positive", "negative"}.issubset(df.columns):
    total = df["positive"] + df["negative"]
    mask = total > 0
    before = df.shape[0]
    df = df[mask].copy()
    print(f"Rows after removing zero-review games: {before} -> {df.shape[0]}")


# keep only top-5 tags by count

def keep_top5_tags_str(s):
    if pd.isna(s):
        return s
    s = str(s).strip()
    if s in ("", "[]", "{}"):
        return s
    try:
        obj = ast.literal_eval(s)
    except Exception:
        return s

    if isinstance(obj, dict):
        # take top 5
        items = sorted(obj.items(), key=lambda kv: kv[1], reverse=True)[:5]
        obj_small = dict(items)
        return str(obj_small)
    else:
        return s

if "tags" in df.columns:
    df["tags"] = df["tags"].apply(keep_top5_tags_str)


# Drop columns only one unique value

constant_cols = [c for c in df.columns if df[c].nunique(dropna=True) <= 1]
df = df.drop(columns=constant_cols)
print("Dropped constant columns:", constant_cols)

print("Final shape:", df.shape)
df.head()

After column drop: (89618, 22)
Rows after cleaning estimated_owners: 89618 -> 81200
Rows after removing zero-review games: 81200 -> 72563
Dropped constant columns: []
Final shape: (72563, 22)


,appid,required_age,price,dlc_count,windows,mac,linux,achievements,recommendations,supported_languages,...,publishers,categories,genres,user_score,positive,negative,estimated_owners,peak_ccu,tags,num_reviews_total
0,730,0,0.00,1,1,0,1,1,4401572,"['Czech', 'Danish', 'Dutch', 'English', 'Finni...",...,['Valve'],"['Multi-player', 'Cross-Platform Multiplayer',...","['Action', 'Free To Play']",0,7480813,1135108,150000000.0,1212356,"{'FPS': 90857, 'Shooter': 65397, 'Multiplayer'...",8632939.0
1,578080,0,0.00,0,1,0,0,37,1732007,"['English', 'Korean', 'Simplified Chinese', 'F...",...,"['KRAFTON, Inc.']","['Multi-player', 'PvP', 'Online PvP', 'Stats',...","['Action', 'Adventure', 'Massively Multiplayer...",0,1487960,1024436,75000000.0,616738,"{'Survival': 14838, 'Shooter': 12727, 'Battle ...",2513842.0
2,570,0,0.00,2,1,1,1,0,14337,"['Bulgarian', 'Czech', 'Danish', 'Dutch', 'Eng...",...,['Valve'],"['Multi-player', 'Co-op', 'Steam Trading Cards...","['Action', 'Strategy', 'Free To Play']",0,1998462,451338,350000000.0,555977,"{'Free to Play': 59933, 'MOBA': 20158, 'Multip...",2452595.0
3,271590,17,0.00,0,1,0,0,77,1803063,"['English', 'French', 'Italian', 'German', 'Sp...",...,['Rockstar Games'],"['Single-player', 'Multi-player', 'PvP', 'Onli...","['Action', 'Adventure']",0,1719950,250012,75000000.0,117698,"{'Open World': 32644, 'Action': 23539, 'Multip...",1803832.0
4,359550,17,3.99,9,1,0,0,0,1165929,"['English', 'French', 'Italian', 'German', 'Sp...",...,['Ubisoft'],"['Single-player', 'Multi-player', 'PvP', 'Onli...",['Action'],0,1152763,218446,35000000.0,89916,"{'FPS': 9831, 'PvP': 9162, 'e-sports': 9072, '...",1168020.0


In [42]:
out_path = "/content/games_march2025_cleaned_NEW.csv"
df.to_csv(out_path, index=False)
print("Saved cleaned file to:", out_path)

Saved cleaned file to: /content/games_march2025_cleaned_NEW.csv
